In [51]:
!pip install -q -U kaggle_environments

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [53]:
%%writefile rock_agent.py
#только камень
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [54]:
%%writefile paper_agent.py
# только бумага
def paper(observation, configuration):
    return 1

Overwriting paper_agent.py


In [55]:
%%writefile scissors_agent.py
# только ножницы
def scissors(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [70]:
%%writefile random_rnp.py
import random
# слуйчайно выбирает между камнем и бумагой
def random_rnp(observation, configuration):
    return random.choice([0, 1])


Writing random_rnp.py


In [69]:
%%writefile random_rns.py
import random
# слуйчайно выбирает между камнем и ножницами
def random_rns(observation, configuration):
    return random.choice([0, 2])

Writing random_rns.py


In [68]:
%%writefile random_pns.py
import random
# слуйчайно выбирает между бумагой и ножницами
def random_pns(observation, configuration):
    return random.choice([1, 2])

Writing random_pns.py


In [73]:
%%writefile random_rnpns.py
import random
# слуйчайно выбирает между камнем, бумагой и ножницами
def random_rnpns(observation, configuration):
    return random.choice([0, 1, 2])

Writing random_rnpns.py


In [97]:
%%writefile parity_rock.py
# если ход четный то камень, если нечетный бумага
def parity_rock(observation, configuration):
    if observation.step % 2 == 0:
        return 0
    else:
        return 1

Overwriting parity_rock.py


In [75]:
%%writefile parity_paper.py
# если ход четный то бумага, если нечетный ножницы
def parity_paper(observation, configuration):
    if observation.step % 2 == 0:
        return 1
    else:
        return 2

Writing parity_paper.py


In [99]:
%%writefile parity_scissors.py
# если ход четный то ножницы, если нечетный бумага
def parity_scissors(observation, configuration):
    if observation.step % 2 == 0:
        return 2
    else:
        return 1

Writing parity_scissors.py


In [110]:
%%writefile all.py
# чередует каждый ход с камня-бумага-ножницы
def all(observation, configuration):
    return (observation.step % 3)


Writing all.py


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [111]:
%%writefile last_step.py
def last_step(observation, configuration):
# повторяет последний ход оппонента
    return observation.lastOpponentAction if observation.step > 0 else 0

Writing last_step.py


In [119]:
%%writefile all_second_step.py
def all_second_step(observation, configuration):
# чередует каждый второй ход все действия
    return (observation.step // 2) % 3

Writing all_second_step.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [117]:
#функция на проверку работоспособности игроков
agent= ["last_step.py","random_pns.py"]
res = evaluate(
    "rps", #environment to use - no need to change
    [agent[0], agent[1]], #agents to evaluate
    configuration={"episodeSteps": 100},
    debug=True
)
print(res)

[[0, 0]]


In [123]:
#Симулирует матч между двумя игроками и возвращает победителя
def play_match(player1, player2):
    res = evaluate("rps", #environment to use - no need to change
    [player1, player2], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
    )
    # Создаем таблицу где записываем результат матча для двух игроков, при победе присуждается 1 очко, при ничьей 0.5 каждому и о при проигрыше
    table_win = {str(player1): 1 if res[0][0] > res[0][1] else (0.5 if res[0][0] == res[0][1] else 0),
                 str(player2): 1 if res[0][1] > res[0][0] else (0.5 if res[0][1] == res[0][0] else 0)}
    # Возвращем словарь
    return table_win

# Инициализация функции для турнирнйо сетки, чтобы игроки не играли сами с собой
def tournament(players):
    # Создаем список матчей
    winners = []
    # Инициализируем цикл подобно шахмотной таблице турнира
    for i in range(len(players)):
        # Инициализируем цикл для определение пар игроков, так чтобы не было матчей с самим собой
        for k in range(i + 1, len(players)):
            # Запускаем матч и добаваляем результаты в список матчей
            winner = play_match(players[i], players[k])
            winners.append(winner)
    # print(winners) вывод всех матчей
    return winners # возвращаем список

# Инициализация функции для запуска турнира
def table(players):
    # Запускаем сетку турнира и возвращаем результаты в переменную tab
    tab = tournament(players)
    # Создаем словарь где будет храниться данные по каждому матчу
    results = {}

    # Инициализация результатов для каждого игрока через цикл и задаем начальное значение 0 для каждого игрока
    for player in players:
        results[player] = 0
    # Подсчет побед для каждого игрока через цикл
    for match_ in tab:
    # Суммируем очки с помощью цикла по списку и записываем в результаты для каждого игрока
        for player, score in match_.items():
            results[player] += score
    # Выводим спиок игроков и их очков
    print("Таблица результатов:")
    for player, score in results.items():
        print(f"{player}: {score} очки")

# Вносим список участников по названию созданных ранее функций
players = ["scissors_agent.py", "paper_agent.py", "rock_agent.py","random_rnpns.py","random_pns.py",
           "random_rns.py","random_rnp.py","parity_scissors.py","parity_paper.py","parity_rock.py","all.py","last_step.py","all_second_step.py"]

# Запуск турнира
table(players)

Таблица результатов:
scissors_agent.py: 6.5 очки
paper_agent.py: 5.0 очки
rock_agent.py: 6.5 очки
random_rnpns.py: 6.5 очки
random_pns.py: 5.0 очки
random_rns.py: 7.0 очки
random_rnp.py: 6.0 очки
parity_scissors.py: 5.5 очки
parity_paper.py: 6.0 очки
parity_rock.py: 6.0 очки
all.py: 6.5 очки
last_step.py: 5.0 очки
all_second_step.py: 6.5 очки
